About the Dataset
  1. id : Unique id for a new article
  2. title : The title of a news article
  3. author : Author of a news article
  4. text : The text of the article, could be incomplete
  5. label : A Label that marks whether the news article is real or fake

      

*   **1 : Fake News**
*   **2 : Real News**



Importing The Dependencies

In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# Printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [9]:
# Loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv')

In [10]:
news_dataset.shape

(20999, 5)

In [12]:
# Print the 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
# Counting the number of missing value in the dataset
news_dataset.isnull().sum()

id           0
title      562
author    1973
text        39
label        0
dtype: int64

In [14]:
# Replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [15]:
# Merging the author name and news title 
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [16]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20994    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20995    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20996    Michael J. de la Merced and Rachel Abrams Macy...
20997    Alex Ansary NATO, Russia To Hold Parallel Exer...
20998              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20999, dtype: object


In [19]:
# Separating the data and label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [20]:
print(X)
print(Y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20994  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20995  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20996  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20997  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20998  20799  ...            David Swanson What Keeps the F-35 Alive

[20999 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20994    0
20995    0
20996    0
20997    1
20998    1
Name: label, Length: 2099

Stemming :    

Stemming is the process of reducing a word to its Root word

example :    
actor, actress, acting --> act

In [21]:
port_stem = PorterStemmer()

In [24]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower() 
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [25]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [26]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20994    jerom hudson rapper trump poster child white s...
20995    benjamin hoffman n f l playoff schedul matchup...
20996    michael j de la merc rachel abram maci said re...
20997    alex ansari nato russia hold parallel exercis ...
20998                            david swanson keep f aliv
Name: content, Length: 20999, dtype: object


In [27]:
# Separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [28]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [29]:
print(Y)

[1 0 1 ... 0 1 1]


In [30]:
Y.shape

(20999,)

In [31]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [32]:
print(X)

  (0, 15686)	0.2850197763984485
  (0, 13473)	0.25647071470545285
  (0, 8909)	0.36370037073947287
  (0, 8630)	0.291573687188683
  (0, 7692)	0.24754579485071557
  (0, 7005)	0.2187094043739489
  (0, 4973)	0.23316843483792904
  (0, 3792)	0.2707142344139613
  (0, 3600)	0.3600009721823952
  (0, 2959)	0.2470455977561568
  (0, 2483)	0.36775260337664795
  (0, 267)	0.27028259352234113
  (1, 16799)	0.3011323927720925
  (1, 6816)	0.1904284402339991
  (1, 5503)	0.7151367138253379
  (1, 3568)	0.263477035056066
  (1, 2813)	0.19090350944621187
  (1, 2223)	0.38131035180044764
  (1, 1894)	0.1551361529273462
  (1, 1497)	0.2937799575471476
  (2, 15611)	0.41534333989435995
  (2, 9620)	0.493942770574391
  (2, 5968)	0.3475381049293185
  (2, 5389)	0.386756267798861
  (2, 3103)	0.46030463888565165
  :	:
  (20996, 13122)	0.24720876463658228
  (20996, 12344)	0.2729825550652414
  (20996, 12138)	0.2476674063152218
  (20996, 10306)	0.08034752417435242
  (20996, 9588)	0.17437011793188265
  (20996, 9518)	0.2957673519

Splitting the dataset to training and test data

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state=2)

Training The Model = Logistic Regression

In [34]:
model = LogisticRegression()

In [35]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Evaluation

Accuracy Score

In [37]:
# Accuracy Score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [38]:
print('Accuracy Score of The Training Data : ', training_data_accuracy)

Accuracy Score of The Training Data :  0.9869635097327222


In [39]:
# Accuracy Score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [40]:
print('Accuracy Score of The Testing Data : ', test_data_accuracy)

Accuracy Score of The Testing Data :  0.9764285714285714


Making a Predictive System

In [41]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')


[1]
The news is Fake


In [43]:
print(Y_test[0])

1
